## Introduction

The notebook explores the first ten highest stocks with high MarketCapitalization(largest stocks). Long short Term Memory model performance was used to explore these 10 stocks.

## Objectives
The goal of this notebook is to develop an LSTM model that attempts to predict the 10 stocks for our stakeholder.


## Importation of all relevant packages

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
import seaborn as sns
import plotly.express as px
import altair as alt
import numpy as np
from math import floor,ceil,sqrt
import sys
import datetime as dt
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error

#if not sys.warnoptions:
    #warnings.simplefilter("ignore")
from sklearn.linear_model import LinearRegression
from pmdarima.arima import auto_arima
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from fbprophet import Prophet
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

from ipywidgets import HTML
from io import BytesIO
import base64

import warnings
warnings.simplefilter("ignore")

# Turn off the max column width so the images won't be truncated
pd.set_option('display.max_colwidth', None)
# Show all Columns
pd.set_option('display.max_columns', None)
 
# Turning off the max column will display all the data
# if gathering into sets / array we might want to restrict to a few items
pd.set_option('display.max_seq_items', 3)

## Load Data

In [2]:
#For local path
path = "../data/train_files"
path_stocklist = "../data"

#### DataSet(Preprocessed)

This data has not been processed from other notebooks

In [3]:
%%time

df_prices = pd.read_csv(os.path.join(path, 'stock_prices.csv'))
df_prices_sec = pd.read_csv(os.path.join(path, 'secondary_stock_prices.csv'))
df_fins = pd.read_csv(os.path.join(path, 'financials.csv'))
df_opts = pd.read_csv(os.path.join(path, 'options.csv'))
df_trades = pd.read_csv(os.path.join(path, 'trades.csv'))
stock_list = pd.read_csv(os.path.join(path_stocklist, "stock_list.csv"))

CPU times: user 7.89 s, sys: 1.72 s, total: 9.61 s
Wall time: 10.6 s


#### DataSet(Processed)

This dataset has been processed from other notebooks(feather Engineering Notebook)

In [4]:
df = pd.read_csv(os.path.join(path, 'out.csv'))

In [5]:
stock_list.head(2)

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,1.092828e+07,3.365911e+10,True
1,1305,20211230,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2097.0,3.634636e+09,7.621831e+12,False


### Sorting dataset for the Highest MarketCapitalization

In [6]:
df_stock_list = stock_list.sort_values(by='MarketCapitalization', ascending=False)
df_stock_list.head(10)

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
137,1557,20211230,SPDR S&P500 ETF Trust,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,54920.0,9.503321e+08,5.219224e+13,False
3065,7203,20211230,TOYOTA MOTOR CORPORATION,First Section (Domestic),Prime Market,3700,Transportation Equipment,6,AUTOMOBILES & TRANSPORTATION EQUIPMENT,1,TOPIX Core30,20211230.0,2105.5,1.631499e+10,3.435121e+13,True
2804,6758,20211230,SONY GROUP CORPORATION,First Section (Domestic),Prime Market,3650,Electric Appliances,9,ELECTRIC APPLIANCES & PRECISION INSTRUMENTS,1,TOPIX Core30,20211230.0,14475.0,1.261082e+09,1.825416e+13,True
2867,6861,20211230,KEYENCE CORPORATION,First Section (Domestic),Prime Market,3650,Electric Appliances,9,ELECTRIC APPLIANCES & PRECISION INSTRUMENTS,1,TOPIX Core30,20211230.0,72280.0,2.432077e+08,1.757905e+13,True
2,1306,20211230,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2073.5,7.917718e+09,1.641739e+13,False
2375,6098,20211230,"Recruit Holdings Co.,Ltd.",First Section (Domestic),Prime Market,9050,Services,10,"IT & SERVICES, OTHERS",1,TOPIX Core30,20211230.0,6972.0,1.695960e+09,1.182423e+13,True
4158,9432,20211230,NIPPON TELEGRAPH AND TELEPHONE CORPORATION,First Section (Domestic),Prime Market,5250,Information & Communication,10,"IT & SERVICES, OTHERS",1,TOPIX Core30,20211230.0,3150.0,3.622013e+09,1.140934e+13,True
3577,8035,20211230,Tokyo Electron Limited,First Section (Domestic),Prime Market,3650,Electric Appliances,9,ELECTRIC APPLIANCES & PRECISION INSTRUMENTS,1,TOPIX Core30,20211230.0,66280.0,1.572109e+08,1.041994e+13,True
4404,9984,20211230,SoftBank Group Corp.,First Section (Domestic),Prime Market,5250,Information & Communication,10,"IT & SERVICES, OTHERS",1,TOPIX Core30,20211230.0,5434.0,1.722954e+09,9.362531e+12,True
1508,4063,20211230,"Shin-Etsu Chemical Co.,Ltd.",First Section (Domestic),Prime Market,3200,Chemicals,4,RAW MATERIALS & CHEMICALS,1,TOPIX Core30,20211230.0,19920.0,4.166628e+08,8.299923e+12,True


### Sorting on stocks with high MarketCapitalization in Processed Dataset

In [7]:
df_b = df.loc[df['SecuritiesCode'].isin([1557, 7203, 6758, 6861, 1306, 6098, 9432,8035,9984,4063])]
df_b.head(2)


,Date,SecuritiesCode,ExpectedDividend,Open,High,Low,Close,Volume,Target,BOP,Cpos,bsforce,Opos,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,others_dr,others_dlr,others_cr,0,1
645165,2017-01-04,4063,15.0,9158.0,9447.0,9131.0,9447.0,1447500.0,0.014949,-0.914557,0.50000,723750.0,-0.414557,9341.666667,9447.0,316.0,-289.0,0.031557,86515626.0,86260557.0,9.153558,0.031069,174.995952,-0.020575,-5.881328,3,0,0,1,0,0,7471.4,7022.35,6530.14,5717.11,4303.312,7788.112631,7098.722595,6703.454852,5804.033006,4648.458007,-7.174367e+09,8.426501e+10,0.075297,5.179882e+08,5.182618e+07,3.540187e+06,547467.387532,7094.897302,54.822636,1.028498e-90,7022.35,8157.305378,5887.394622,32.324090,1.568170,1.0,0.0,7190.5,7446.1,6934.9,7.109380,4.914124,1.0,0.0,6570.0,9447.0,8008.5,40.969191,1.00000,480.231534,2.108391,321.990364,183.484411,138.505952,7131.416667,6981.038462,7311.356469,6989.366105,0.948779,0.872631,0.076148,0.403932,25.435846,-422.35,128.298959,105.43308,22.865879,8078.5,8003.5,8041.0,7433.5,37.165663,16.781475,57.278576,12.041392,626.306914,6195.0,6125.0,100.0,56.0,44.0,1.0,0.0,82.634276,1.000000,0.528564,0.515650,30.705758,75.468691,100.000000,60.493827,-0.000000,666.947059,39.131075,4.606861,2.694727,1.912133,10.627269,8.592798,2.034471,34.764622,29.835953,244.529540,1.0,0.0
645166,2017-01-05,4063,15.0,9435.0,9437.0,9277.0,9298.0,1714800.0,-0.011020,0.856250,-0.36875,-632332.5,0.487500,9337.333333,9298.0,160.0,137.0,-0.014520,87726630.0,87547049.0,9.137662,-0.014627,86.167956,-0.050715,-5.702605,4,0,0,0,1,0,7917.0,7149.75,6580.90,5739.05,4316.836,8291.408421,7308.177586,6805.201721,5838.798846,4667.019053,-7.175632e+09,8.426329e+10,0.053899,4.074891e+08,6.344763e+05,3.643439e+06,476171.729607,7286.222846,51.583068,1.028498e-90,7149.75,8647.776151,5651.723849,41.904295,1.217027,1.0,0.0,7429.9,7680.5,7179.3,6.745717,4.227255,1.0,0.0,6570.0,9447.0,8008.5,40.239169,0.94821,449.208381,2.108217,456.617912,238.111112,218.506800,7356.250000,7078.653846,7616.993935,7160.376023,0.989054,0.874123,0.114931,0.504073,25.542831,-289.75,154.367531,110.73730,43.630232,8078.5,8003.5,8041.0,7433.5,58.593485,20.244121,55.509057,11.669395,331.262674,6195.0,6125.0,96.0,52.0,44.0,0.0,0.0,79.508351,0.903387,0.815878,0.599511,38.061629,74.693691,94.820994,83.047327,-5.179006,995.023529,40.878788,6.377010,3.431184,2.945826,10.754164,9.025071,1.729093,-1.577220,-1.589791,239.095551,1.0,0.0


### Predicting stocks for using LTMG

In [8]:
# Selecting the code with the highest MarketCapitalization
code1 = df_b['SecuritiesCode'].unique()

In [9]:
# We will write a lstm function that returns the Root mean squared error(RMSE)

def lstm_prediction(df_b):
    
    # Choose Securitie and Testsize in Days

    # window size(prediction)
    testsize = 30


    #adjusting Models to Testsize
    Size = (len(df_b.index)-testsize)/len(df_b.index)

    # reshaping dataset
    shape=df_b.shape[0]
    df_new=df_b[['Close']]
    df_new.head()
    train_set=df_new.iloc[:ceil(shape*Size)]
    valid_set=df_new.iloc[ceil(shape*Size):]

    print('Training Size',train_set.shape)
    print('Test Size',valid_set.shape)
    
    dataset = df_new.values
    train=train_set
    valid=valid_set
    
    # Scalling the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)
    
    # splitting into x and y train
    x_train, y_train = [], []
    
    #
    for i in range(40,len(train)):
        x_train.append(scaled_data[i-40:i,0])
        y_train.append(scaled_data[i,0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
    
    # Model design network
    model = Sequential()

    # Units == number of Neurons
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # fit network
    model.fit(x_train, y_train,epochs=10, batch_size=1, verbose=2)
    
    
    inputs = df_new[len(df_new) - len(valid) - 40:].values
    inputs = inputs.reshape(-1,1)
    inputs  = scaler.transform(inputs)
    
    # looping through test data
    X_test = []
    for i in range(40,inputs.shape[0]):
        X_test.append(inputs[i-40:i,0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
    
    # prediction
    closing_price = model.predict(X_test)
    # invert scaling for forecast
    closing_price = scaler.inverse_transform(closing_price)
    
    # model Evaluation
    rmse = mean_squared_error(valid,closing_price,squared=False)
    mape =mean_absolute_percentage_error(valid,closing_price)
    print('RMSE,MAPE value on validation set:',rmse,mape)
    print('-----------------------------------------------------------')
    print('-----------------------------------------------------------')

    
   
    
    return {"rmse":rmse,"mape":mape}

We have created a function that returns the rmse and mape . We will create a loop that iterates through the 10 stocks and returns their rmse and mape values

In [10]:
err = {}
for scode in code1:
        
        df_pred = df.loc[df['SecuritiesCode'] == scode]
        scode_err = lstm_prediction(df_pred)
        err[scode] = scode_err

Training Size (1172, 1)
Test Size (30, 1)
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-06-15 10:12:32.769339: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-15 10:12:32.771817: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2022-06-15 10:12:33.372265: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-15 10:12:34.430128: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:12:34.596456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:12:34.687893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:12:34.763562: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:12:34.904119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1132/1132 - 20s - loss: 0.0032 - 20s/epoch - 18ms/step
Epoch 2/10
1132/1132 - 19s - loss: 0.0010 - 19s/epoch - 17ms/step
Epoch 3/10
1132/1132 - 19s - loss: 8.8452e-04 - 19s/epoch - 17ms/step
Epoch 4/10
1132/1132 - 19s - loss: 7.7324e-04 - 19s/epoch - 17ms/step
Epoch 5/10
1132/1132 - 20s - loss: 6.3309e-04 - 20s/epoch - 18ms/step
Epoch 6/10
1132/1132 - 20s - loss: 6.0308e-04 - 20s/epoch - 17ms/step
Epoch 7/10
1132/1132 - 20s - loss: 5.6883e-04 - 20s/epoch - 17ms/step
Epoch 8/10
1132/1132 - 20s - loss: 4.9071e-04 - 20s/epoch - 18ms/step
Epoch 9/10
1132/1132 - 22s - loss: 5.6136e-04 - 22s/epoch - 20ms/step
Epoch 10/10
1132/1132 - 21s - loss: 4.5605e-04 - 21s/epoch - 19ms/step


...

RMSE,MAPE value on validation set: 72.63000946740333 0.018489479078750995
-----------------------------------------------------------
-----------------------------------------------------------
Training Size (1172, 1)
Test Size (30, 1)
Epoch 1/10


2022-06-15 10:36:58.324066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:36:58.490490: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:36:58.566604: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:36:58.639494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:36:58.738336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1132/1132 - 23s - loss: 0.0034 - 23s/epoch - 20ms/step
Epoch 2/10
1132/1132 - 22s - loss: 0.0011 - 22s/epoch - 19ms/step
Epoch 3/10
1132/1132 - 23s - loss: 9.0081e-04 - 23s/epoch - 21ms/step
Epoch 4/10
1132/1132 - 24s - loss: 7.9739e-04 - 24s/epoch - 21ms/step
Epoch 5/10
1132/1132 - 23s - loss: 6.6585e-04 - 23s/epoch - 20ms/step
Epoch 6/10
1132/1132 - 24s - loss: 5.8651e-04 - 24s/epoch - 21ms/step
Epoch 7/10
1132/1132 - 25s - loss: 6.1831e-04 - 25s/epoch - 22ms/step
Epoch 8/10
1132/1132 - 27s - loss: 5.7079e-04 - 27s/epoch - 24ms/step
Epoch 9/10
1132/1132 - 25s - loss: 5.2906e-04 - 25s/epoch - 22ms/step
Epoch 10/10
1132/1132 - 29s - loss: 5.5091e-04 - 29s/epoch - 26ms/step


2022-06-15 10:41:02.356377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:41:02.449722: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-15 10:41:02.555111: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


RMSE,MAPE value on validation set: 189.28152764480396 0.0219067187710767
-----------------------------------------------------------
-----------------------------------------------------------


In [11]:
err

{4063: {'rmse': 466.51479585331595, 'mape': 0.019727048826003887},
 6098: {'rmse': 257.2924575399272, 'mape': 0.028750763123246006},
 6758: {'rmse': 299.36099790863966, 'mape': 0.01757005704109525},
 6861: {'rmse': 1397.1538301081089, 'mape': 0.01441522800221872},
 7203: {'rmse': 28.6534728306707, 'mape': 0.011466692713091219},
 8035: {'rmse': 1708.091456346741, 'mape': 0.02597358034100416},
 9432: {'rmse': 72.63000946740333, 'mape': 0.018489479078750995},
 9984: {'rmse': 189.28152764480396, 'mape': 0.0219067187710767}}

## Conclusion:
We can conclude that from our results that our model performed extremely well in stock 7203. this may be true because it a Toyota stock company, they have a well structured company that could be easily.